In [ ]:
vehicle_data = [
     {"x": 3, "y": 1, "length": 2, "orientation": "H", "movable": True},  # 목표 차량 (빨간색)
     {"x": 4, "y": 1, "length": 3, "orientation": "V", "movable": True},
     {"x": 4, "y": 2, "length": 3, "orientation": "H", "movable": True},
     {"x": 1, "y": 4, "length": 3, "orientation": "V", "movable": True},
]

In [ ]:
# 단순합

import gym
import numpy as np
import pygame
from gym import spaces
from collections import deque
import numpy as np
import pygame
from gym import spaces

class Vehicle:
    def __init__(self, x, y, length, orientation, movable=True):
        self.x = x
        self.y = y
        self.length = length
        self.orientation = orientation
        self.movable = movable

    def get_positions(self):
        positions = []
        if self.orientation == 'H':
            positions = [(self.x, self.y + i) for i in range(self.length)]
        elif self.orientation == 'V':
            positions = [(self.x + i, self.y) for i in range(self.length)]
        return positions

class RushHourEnv(gym.Env):
    def __init__(self, vehicle_data, grid_size=(8, 8), max_steps=1000):
        super(RushHourEnv, self).__init__()
        self.grid_size = grid_size
        self.max_steps = max_steps
        self.steps = 0
        self.window_size = 400
        self.cell_size = self.window_size // self.grid_size[0]
        self.screen = None

        self.vehicle_data = vehicle_data
        self.vehicles = [
            Vehicle(data["x"], data["y"], data["length"], data["orientation"], data["movable"])
            for data in self.vehicle_data
        ]
        self.target_vehicle = self.vehicles[0]
        self.exit_wall = (3, self.grid_size[1] - 1)
        self.previous_distance = self.grid_size[1] - self.target_vehicle.get_positions()[-1][1]
        self.walls = self.create_walls()

        self.action_space = spaces.MultiDiscrete([len(self.vehicles), 2])
        self.observation_space = spaces.Box(low=0, high=1, shape=self.grid_size, dtype=np.int32)

        self.move_log = []  # 이동 경로 로그

    def reset(self):
        self.steps = 0
        self.vehicles = [
            Vehicle(data["x"], data["y"], data["length"], data["orientation"], data["movable"])
            for data in self.vehicle_data
        ]
        self.target_vehicle = self.vehicles[0]
        self.previous_distance = self.grid_size[1] - self.target_vehicle.get_positions()[-1][1]
        self.move_log = []  # 초기화 시 이동 로그 초기화
        return self.get_observation()

    def step(self, actions):
        rewards = [0] * len(self.vehicles)
        done = False

        for vehicle_idx, direction in actions:
            vehicle = self.vehicles[vehicle_idx]
            if vehicle.movable and self.move_vehicle(vehicle, direction):
                # 이동 로그 추가
                self.move_log.append((vehicle_idx, direction))
                if vehicle == self.target_vehicle:
                    target_front_position = self.target_vehicle.get_positions()[-1]
                    new_distance = self.grid_size[1] - target_front_position[1]
                    if new_distance < self.previous_distance:
                        rewards[vehicle_idx] += 10
                    self.previous_distance = new_distance
                elif self.is_target_unblocked():
                    rewards[vehicle_idx] += 5

        target_front_position = self.target_vehicle.get_positions()[-1]
        if target_front_position == (self.exit_wall[0], self.exit_wall[1] - 1):
            rewards[0] += 100
            done = True
        elif self.steps >= self.max_steps:
            done = True

        self.steps += 1
        target_vehicle_reward = rewards[0]
        other_vehicles_reward = sum(rewards) - target_vehicle_reward
        total_reward = target_vehicle_reward + 0.5 * other_vehicles_reward
        return self.get_observation(), total_reward, done, {}

    def get_move_log(self):
        """이동 경로 데이터를 반환"""
        return self.move_log

    def create_walls(self):
        walls = set()
        for i in range(self.grid_size[0]):
            walls.add((i, 0))
            walls.add((i, self.grid_size[1] - 1))
        for j in range(self.grid_size[1]):
            walls.add((0, j))
            walls.add((self.grid_size[0] - 1, j))
        return walls

    def get_observation(self):
        grid = np.zeros(self.grid_size, dtype=np.int32)
        for i, vehicle in enumerate(self.vehicles, start=1):
            for pos in vehicle.get_positions():
                if 0 <= pos[0] < self.grid_size[0] and 0 <= pos[1] < self.grid_size[1]:
                    grid[pos] = i
        for wall in self.walls:
            grid[wall] = -1
        target_front_position = self.target_vehicle.get_positions()[-1]
        grid[target_front_position] = 9  # Highlight the target vehicle's front
        return grid

    def is_target_unblocked(self):
        """Check if the target vehicle's path is unblocked."""
        target_positions = self.target_vehicle.get_positions()
        target_x, target_y = target_positions[-1]
        for y in range(target_y + 1, self.grid_size[1]):
            if any(vehicle.get_positions() for vehicle in self.vehicles if (target_x, y) in vehicle.get_positions()):
                return False
        return True


    def move_vehicle(self, vehicle, direction):
        step = 1 if direction == 1 else -1
        if vehicle.orientation == 'H':
            for i in range(vehicle.length):
                new_y = vehicle.y + step
                new_pos = (vehicle.x, new_y + i)
                if not self.is_valid_position(vehicle, new_pos):
                    return False
            vehicle.y += step
        elif vehicle.orientation == 'V':
            for i in range(vehicle.length):
                new_x = vehicle.x + step
                new_pos = (new_x + i, vehicle.y)
                if not self.is_valid_position(vehicle, new_pos):
                    return False
            vehicle.x += step
        return True

    def is_valid_position(self, vehicle, position):
        x, y = position
        if position in self.walls:
            return False
        if not (0 <= x < self.grid_size[0] and 0 <= y < self.grid_size[1]):
            return False
        for other_vehicle in self.vehicles:
            if other_vehicle != vehicle and position in other_vehicle.get_positions():
                return False
        return True

    def render(self, mode='human', save_as_image=True):
        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.window_size, self.window_size))
            pygame.display.set_caption("Rush Hour Environment")

        self.screen.fill((255, 255, 255))

        for i, vehicle in enumerate(self.vehicles):
            color = self.vehicle_colors[i]
            for pos in vehicle.get_positions():
                rect = pygame.Rect(pos[1] * self.cell_size, pos[0] * self.cell_size, self.cell_size, self.cell_size)
                pygame.draw.rect(self.screen, color, rect)

        for wall in self.walls:
            rect = pygame.Rect(wall[1] * self.cell_size, wall[0] * self.cell_size, self.cell_size, self.cell_size)
            color = (0, 255, 0) if wall == self.exit_wall else (0, 0, 0)
            pygame.draw.rect(self.screen, color, rect)

        pygame.display.flip()

    def close(self):
        if self.screen is not None:
            pygame.quit()
            self.screen = None

In [ ]:
# 가중합
class RushHourEnv(gym.Env):
    def __init__(self, vehicle_data, grid_size=(8, 8), max_steps=1000):
        super(RushHourEnv, self).__init__()
        self.grid_size = grid_size
        self.max_steps = max_steps
        self.steps = 0
        self.window_size = 400
        self.cell_size = self.window_size // self.grid_size[0]
        self.screen = None

        self.vehicle_data = vehicle_data
        self.vehicles = [
            Vehicle(data["x"], data["y"], data["length"], data["orientation"], data["movable"])
            for data in self.vehicle_data
        ]
        self.target_vehicle = self.vehicles[0]
        self.exit_wall = (3, self.grid_size[1] - 1)
        self.previous_distance = self.grid_size[1] - self.target_vehicle.get_positions()[-1][1]
        self.walls = self.create_walls()

        self.action_space = spaces.MultiDiscrete([len(self.vehicles), 2])
        self.observation_space = spaces.Box(low=0, high=1, shape=self.grid_size, dtype=np.int32)

        # 색상 팔레트 생성 (RGB 값)
        self.vehicle_colors = [(255, 0, 0)] + [
            (np.random.randint(50, 256), np.random.randint(50, 256), np.random.randint(50, 256))
            for _ in range(1, len(self.vehicles))
        ]

    def create_walls(self):
        walls = set()
        for i in range(self.grid_size[0]):
            walls.add((i, 0))
            walls.add((i, self.grid_size[1] - 1))
        for j in range(self.grid_size[1]):
            walls.add((0, j))
            walls.add((self.grid_size[0] - 1, j))
        return walls

    def reset(self):
        self.steps = 0
        self.vehicles = [
            Vehicle(data["x"], data["y"], data["length"], data["orientation"], data["movable"])
            for data in self.vehicle_data
        ]
        self.target_vehicle = self.vehicles[0]
        self.previous_distance = self.grid_size[1] - self.target_vehicle.get_positions()[-1][1]
        return self.get_observation()

    def get_observation(self):
        grid = np.zeros(self.grid_size, dtype=np.int32)
        for i, vehicle in enumerate(self.vehicles, start=1):
            for pos in vehicle.get_positions():
                if 0 <= pos[0] < self.grid_size[0] and 0 <= pos[1] < self.grid_size[1]:
                    grid[pos] = i
        for wall in self.walls:
            grid[wall] = -1
        target_front_position = self.target_vehicle.get_positions()[-1]
        grid[target_front_position] = 9  # Highlight the target vehicle's front
        return grid

    def is_target_unblocked(self):
        """Check if the target vehicle's path is unblocked."""
        target_positions = self.target_vehicle.get_positions()
        target_x, target_y = target_positions[-1]
        for y in range(target_y + 1, self.grid_size[1]):
            if any(vehicle.get_positions() for vehicle in self.vehicles if (target_x, y) in vehicle.get_positions()):
                return False
        return True

    def step(self, actions):
        rewards = [0] * len(self.vehicles)
        done = False

        for vehicle_idx, direction in actions:
            vehicle = self.vehicles[vehicle_idx]
            rewards[vehicle_idx] -= 1
            if vehicle.movable and self.move_vehicle(vehicle, direction):
                if vehicle == self.target_vehicle:
                    target_front_position = self.target_vehicle.get_positions()[-1]
                    new_distance = self.grid_size[1] - target_front_position[1]
                    if new_distance < self.previous_distance:
                        rewards[vehicle_idx] += 10
                    else:
                        rewards[vehicle_idx] -= 5
                    self.previous_distance = new_distance
                elif self.is_target_unblocked():
                    rewards[vehicle_idx] += 5

        # Check if the target vehicle reached the goal
        target_front_position = self.target_vehicle.get_positions()[-1]
        if target_front_position == (self.exit_wall[0], self.exit_wall[1] - 1):
            rewards[0] += 100
            done = True
        elif self.steps >= self.max_steps:
            done = True

        self.steps += 1
        target_vehicle_reward = rewards[0]
        other_vehicles_reward = sum(rewards) - target_vehicle_reward
        total_reward = target_vehicle_reward + 0.5 * other_vehicles_reward
        return self.get_observation(), total_reward, done, {}

    def move_vehicle(self, vehicle, direction):
        step = 1 if direction == 1 else -1
        if vehicle.orientation == 'H':
            for i in range(vehicle.length):
                new_y = vehicle.y + step
                new_pos = (vehicle.x, new_y + i)
                if not self.is_valid_position(vehicle, new_pos):
                    return False
            vehicle.y += step
        elif vehicle.orientation == 'V':
            for i in range(vehicle.length):
                new_x = vehicle.x + step
                new_pos = (new_x + i, vehicle.y)
                if not self.is_valid_position(vehicle, new_pos):
                    return False
            vehicle.x += step
        return True

    def is_valid_position(self, vehicle, position):
        x, y = position
        if position in self.walls:
            return False
        if not (0 <= x < self.grid_size[0] and 0 <= y < self.grid_size[1]):
            return False
        for other_vehicle in self.vehicles:
            if other_vehicle != vehicle and position in other_vehicle.get_positions():
                return False
        return True

    def render(self, mode='human', save_as_image=True):
        if self.screen is None:
            pygame.init()
            self.screen = pygame.display.set_mode((self.window_size, self.window_size))
            pygame.display.set_caption("Rush Hour Environment")

        self.screen.fill((255, 255, 255))

        for i, vehicle in enumerate(self.vehicles):
            color = self.vehicle_colors[i]
            for pos in vehicle.get_positions():
                rect = pygame.Rect(pos[1] * self.cell_size, pos[0] * self.cell_size, self.cell_size, self.cell_size)
                pygame.draw.rect(self.screen, color, rect)

        for wall in self.walls:
            rect = pygame.Rect(wall[1] * self.cell_size, wall[0] * self.cell_size, self.cell_size, self.cell_size)
            color = (0, 255, 0) if wall == self.exit_wall else (0, 0, 0)
            pygame.draw.rect(self.screen, color, rect)

        pygame.display.flip()

        if save_as_image:
            pygame.image.save(self.screen, "current_frame.png")

    def close(self):
        if self.screen is not None:
            pygame.quit()
            self.screen = None

# rushour env

## 환경 정의

1. 8 by 8 size의 크기로 마지막 layer는 벽으로 구성 되어 있고 차량은 6 by 6 크기에서 움직일 수 있다.

2. 차량의 크기는 2, 3으로 정해져있고 target 차량의 크기는 2로 고정이다.

3. 차량은 수직, 수평으로 생성되며 차량기준 위, 아래로만 움직일 수 있다.

4. 차량은 움직이는 방향에 장애물이 있으면 그 이상의 거리를 움직일 수 없다.

5. 장애물은 벽, 다른 차량을 의미한다.

6. 차량의 생성 수는 초기에 고정된다.

7. episode는 차량이 목표 위치(출구)로 이동하면 종료하게 된다.

## 보상 시스템 (`step` 함수 내 구현)

각 에피소드의 스텝마다, `actions` 매개변수를 통해 각 차량에 대한 행동이 입력됩니다. 이 행동들은 차량이 이동 가능한지를 결정하며, 차량의 이동 여부와 목표 차량의 상태에 따라 보상이 계산됩니다.

### 보상 계산 방법:

- **차량 이동**: 각 차량의 이동 가능 여부를 판단하여, 이동 성공 시 해당 차량의 보상을 업데이트합니다.
- **목표 차량 보상**:
  - `target_vehicle` (목표 차량)이 성공적으로 움직이고, 출구에 더 가까워질 경우, 이 차량에 대한 보상을 증가시킵니다.
  - 목표 차량이 출구 바로 앞에 도달하면 (`exit_wall` 위치), 큰 보상을 부여하고 에피소드를 종료합니다.
- **추가 보상**:
  - 목표 차량의 경로가 다른 차량에 의해 차단되지 않은 경우, 추가 보상을 제공합니다.

### 보상 값:

- **일반 차량 이동**: 이동 성공 시 보상 없음.
- **목표 차량 이동**:
  - 출구에 가까워질 때마다 +10의 보상.
  - 출구에 도달 시 +100의 보상.
- **경로 차단 해제**: 차단되지 않은 경우 +5의 보상.

### 총합 보상 계산:

- `total_reward`: 계산된 각 차량의 보상을 합산하여 에피소드의 총 보상으로 설정합니다.
- 목표 차량의 보상은 다른 차량의 보상과 별도로 계산되며, 전체 보상에 크게 영향을 미칩니다.

각 스텝의 결과로서, 보상의 총합, 에피소드의 종료 여부, 그리고 다음 관측 상태가 반환됩니다. 이 보상 시스템은 플레이어가 목표 차량을 효율적으로 움직여 출구로 빠르게 도달하도록 동기를 부여합니다.

## 각 함수 설명
### `__init__(self, vehicle_data, grid_size=(8, 8), max_steps=1000)`

- **기능**: `RushHourEnv` 클래스의 생성자입니다.
- **매개변수**:
  - `vehicle_data`: 각 차량에 대한 데이터를 포함하는 사전 목록입니다.
  - `grid_size`: 격자의 차원을 나타내는 튜플입니다.
  - `max_steps`: 에피소드가 종료되기 전의 최대 스텝 수입니다.
- **속성**:
  - 게임 환경을 초기화하고, 격자 크기와 최대 스텝 수를 설정하며, 차량 데이터를 로드합니다.
  - 차량을 준비하고 목표 차량 및 출구 벽을 설정합니다.
  - 행동 공간과 관측 공간을 초기화합니다.

### `reset(self)`

환경을 초기 상태로 재설정합니다.

- **반환**: 환경을 재설정한 후의 초기 격자 상태입니다.
- 이동 로그를 지우고 초기 데이터에 따라 차량 위치를 재설정합니다.

### `step(self, actions)`

주어진 행동에 따라 환경에서 한 스텝을 실행합니다.

- **매개변수**:
  - `actions`: 각 튜플이 하나의 차량과 그 차량의 행동을 포함하는 튜플 목록입니다.
- **반환**: 다음 관측, 획득한 보상의 총합, 에피소드 종료 여부 및 추가 정보를 포함하는 튜플입니다.
- 선택된 행동에 따라 차량을 이동시키고, 보상을 업데이트하며, 목표 차량이 출구에 도달하면 에피소드를 종료합니다.

### `get_move_log(self)`

이동 경로 데이터를 반환합니다.

### `create_walls(self)`

환경의 벽을 생성합니다.

- **반환**: 격자 경계에 벽을 설정하는 위치의 집합입니다.

### `get_observation(self)`

현재 환경의 상태를 관측값으로 반환합니다.

- **반환**: 현재 격자 상태를 나타내는 numpy 배열입니다.
- 각 차량의 위치를 격자에 표시하고, 특정 차량의 앞부분을 강조 표시합니다.

### `is_target_unblocked(self)`

목표 차량의 경로가 차단되지 않았는지 확인합니다.

- **반환**: 목표 차량의 경로가 열려 있으면 True, 그렇지 않으면 False를 반환합니다.

### `move_vehicle(self, vehicle, direction)`

주어진 방향으로 차량을 이동시킵니다.

- **매개변수**:
  - `vehicle`: 이동할 차량 객체입니다.
  - `direction`: 이동 방향(1은 앞으로, -1은 뒤로)입니다.
- **반환**: 이동이 가능하면 True, 그렇지 않으면 False를 반환합니다.
- 차량의 이동 가능성을 확인하고, 유효한 위치로 차량을 이동시킵니다.

### `is_valid_position(self, vehicle, position)`

주어진 위치에 차량이 위치할 수 있는지 확인합니다.

- **매개변수**:
  - `vehicle`: 위치를 확인할 차량 객체입니다.
  - `position`: 확인할 위치의 좌표입니다.
- **반환**: 위치가 유효하면 True, 그렇지 않으면 False를 반환합니다.

### `render(self, mode='human', save_as_image=True)`

환경을 시각적으로 렌더링합니다.

- **매개변수**:
  - `mode`: 렌더링 모드입니다.
  - `save_as_image`: 이미지로 저장할지 여부입니다.
- pygame을 사용하여 환경을 윈도우에 그립니다. 선택적으로 이미지 파일로 저장할 수 있습니다.

### `close(self)`

렌더링에 사용된 자원을 정리합니다.

- pygame 윈도우를 닫고 필요한 정리 작업을 수행합니다.


# 차이점: 기여도 단순합 vs 기여도 가중합

## 차이가 있는 부분

### **함수: `step`**

#### 1. **기여도 단순합**
- 총 보상을 계산할 때 모든 차량의 보상을 단순히 합산합니다.
```
return self.get_observation(), sum(rewards), done, {}
```
#### 2. **기여도 가중합**
타겟 차량(rewards[0])의 보상을 별도로 계산하고, 나머지 차량의 보상에 가중치(0.5)를 적용하여 합산합니다.
```
target_vehicle_reward = rewards[0]
other_vehicles_reward = sum(rewards) - target_vehicle_reward
total_reward = target_vehicle_reward + 0.5 * other_vehicles_reward
return self.get_observation(), total_reward, done, {}
```
